Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'fer2013.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (28709, 48, 48) (28709,)
Validation set (3589, 48, 48) (3589,)
Test set (3589, 48, 48) (3589,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 48
num_labels = 7

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (28709, 2304) (28709, 7)
Validation set (3589, 2304) (3589, 7)
Test set (3589, 2304) (3589, 7)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [5]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized


InternalError: Dst tensor is not initialized.
	 [[Node: Const = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [10000,2304] values: [255 255 255]...>, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op u'Const', defined at:
  File "/home/dsvn/anaconda2/envs/tensorflow/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/dsvn/anaconda2/envs/tensorflow/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/dsvn/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/dsvn/anaconda2/envs/tensorflow/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/dsvn/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/dsvn/anaconda2/envs/t

Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [13]:
batch_size = 128
hidden_layer_size = 2048
graph = tf.Graph()

beta = 0.0001


with graph.as_default():
    tf_train_batch = tf.placeholder(shape=(batch_size, image_size * image_size), dtype=tf.float32)
    tf_train_labels = tf.placeholder(shape=(batch_size, num_labels), dtype=tf.float32)
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    bias = tf.Variable(tf.zeros([hidden_layer_size]))
    
    layer1 = tf.matmul(tf_train_batch, weights) + bias
    
    weights_final = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
    bias_final = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(layer1, weights_final) + bias_final
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    # Add L2 regularization
    l2_loss = tf.nn.l2_loss(weights) + tf.nn.l2_loss(bias) + tf.nn.l2_loss(weights_final) + tf.nn.l2_loss(bias_final)
    total_loss = loss + beta * l2_loss
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(total_loss)
    
    train_layer1 = tf.matmul(tf_train_batch, weights) + bias
    train_prediction = tf.nn.softmax(tf.matmul(train_layer1, weights_final) + bias_final)
    
    valid_layer1 = tf.matmul(tf_valid_dataset, weights) + bias
    valid_prediction = tf.nn.softmax(tf.matmul(valid_layer1, weights_final) + bias_final)
    
    test_layer1 = tf.matmul(tf_test_dataset, weights) + bias
    test_prediction = tf.nn.softmax(tf.matmul(test_layer1, weights_final) + bias_final)

writer = tf.train.SummaryWriter('./full_connected', graph=graph)
writer.close()
    
    

Let's run it:

In [14]:
num_steps = 10001
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        
        feed_dict = {tf_train_batch : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, total_loss, train_prediction], feed_dict=feed_dict)
        
        if step % 1000 == 0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
        
        


Minibatch loss at step 0: 222683.031250
Minibatch accuracy: 14.8%
Validation accuracy: 13.7%
Minibatch loss at step 1000: nan
Minibatch accuracy: 11.7%
Validation accuracy: 13.7%
Minibatch loss at step 2000: nan
Minibatch accuracy: 13.3%
Validation accuracy: 13.7%
Minibatch loss at step 3000: nan
Minibatch accuracy: 14.8%
Validation accuracy: 13.7%
Minibatch loss at step 4000: nan
Minibatch accuracy: 9.4%
Validation accuracy: 13.7%
Minibatch loss at step 5000: nan
Minibatch accuracy: 11.7%
Validation accuracy: 13.7%
Minibatch loss at step 6000: nan
Minibatch accuracy: 15.6%
Validation accuracy: 13.7%
Minibatch loss at step 7000: nan
Minibatch accuracy: 10.2%
Validation accuracy: 13.7%
Minibatch loss at step 8000: nan
Minibatch accuracy: 10.9%
Validation accuracy: 13.7%
Minibatch loss at step 9000: nan
Minibatch accuracy: 11.7%
Validation accuracy: 13.7%
Minibatch loss at step 10000: nan
Minibatch accuracy: 11.7%
Validation accuracy: 13.7%
Test accuracy: 13.0%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [8]:
batch_size = 128
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                            shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Layer 1
    
    
    weights_1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases_1 = tf.Variable(tf.zeros([hidden_layer_size]))
  
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    
    # Hidden layer
    
    weights_h = tf.Variable(
                tf.truncated_normal([hidden_layer_size, num_labels]))
    biases_h = tf.Variable(tf.zeros([num_labels]))
    
    logits = tf.matmul(hidden, weights_h) + biases_h
    
    loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
     # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    

    train_prediction = tf.nn.softmax(logits)
  
    valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
    valid_logits = tf.matmul(valid_hidden, weights_h) + biases_h
    valid_prediction = tf.nn.softmax(valid_logits)
    
    test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
    test_logits = tf.matmul(test_hidden, weights_h) + biases_h
    test_prediction = tf.nn.softmax(test_logits)
    
    

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 187437.343750
Minibatch accuracy: 12.5%
Validation accuracy: 24.5%
Minibatch loss at step 500: 1.822498
Minibatch accuracy: 25.8%
Validation accuracy: 24.5%
Minibatch loss at step 1000: 1.809931
Minibatch accuracy: 27.3%
Validation accuracy: 24.5%
Minibatch loss at step 1500: 1.874222
Minibatch accuracy: 20.3%
Validation accuracy: 24.5%
Minibatch loss at step 2000: 1.836932
Minibatch accuracy: 22.7%
Validation accuracy: 24.5%
Minibatch loss at step 2500: 1.839357
Minibatch accuracy: 24.2%
Validation accuracy: 24.5%
Minibatch loss at step 3000: 1.875070
Minibatch accuracy: 21.1%
Validation accuracy: 24.5%
Test accuracy: 24.9%
